In [55]:
import numpy as np
from IPython.display import Audio
import librosa
import librosa.display
from typing import Literal
import random
from pathlib import Path
from tqdm import tqdm
import torchaudio
import torch

In [3]:
from scripts.noises import *

In [65]:
[1, 2, 3, 4][:-1]

[1, 2, 3]

In [89]:
def apply_noise(signal, sr, noise_type: Literal['random', 'white', 'blue', 'violet', 'brownian', 'pink', 'impulse', 'frequency', 'clipping'] = 'random'):
    if noise_type == 'random':
        noise_type = random.choice(['white', 'blue', 'violet', 'brownian', 'pink', 'impulse', 'frequency', 'clipping'])

    if noise_type == 'white':
        noise_signal = white_noise(signal.shape[0])
        noise_signal = (signal[:-1] if len(signal) % 2 == 1 else signal) + noise_signal * 0.001
    elif noise_type == 'blue':
        noise_signal = blue_noise(signal.shape[0])
        noise_signal = (signal[:-1] if len(signal) % 2 == 1 else signal) + noise_signal * 0.001
    elif noise_type == 'violet':
        noise_signal = violet_noise(signal.shape[0])
        noise_signal = (signal[:-1] if len(signal) % 2 == 1 else signal) + noise_signal * 0.001
    elif noise_type == 'brownian':
        noise_signal = brownian_noise(signal.shape[0])
        noise_signal = (signal[:-1] if len(signal) % 2 == 1 else signal) + noise_signal * 0.02
    elif noise_type == 'pink':
        noise_signal = pink_noise(signal.shape[0])
        noise_signal = (signal[:-1] if len(signal) % 2 == 1 else signal) + noise_signal * 0.001
    elif noise_type == 'impulse':
        noise_signal = impulse_noise(signal, low_threshold=0.05, high_threshold=0.5, size=0.0001)
    elif noise_type == 'frequency':
        noise_signal = frequency_noise(signal, sr, min_freq=1000, max_freq=3000, c=0.2)
    elif noise_type == 'clipping':
        noise_signal = clipping_noise(signal, threshold=0.2)

    return noise_signal

In [4]:
file_path = "data/Actor_01/03-01-01-01-01-01-01.wav"

In [7]:
signal, sr = librosa.load(file_path, sr=None)

Audio(signal, rate=sr)

In [72]:
Audio(apply_noise(signal, sr, 'blue'), rate=sr)

In [73]:
Audio(apply_noise(signal, sr, 'violet'), rate=sr)

In [84]:
Audio(apply_noise(signal, sr, 'brownian'), rate=sr)

In [85]:
Audio(apply_noise(signal, sr, 'pink'), rate=sr)

In [86]:
Audio(apply_noise(signal, sr, 'impulse'), rate=sr)

In [90]:
Audio(apply_noise(signal, sr, 'frequency'), rate=sr)

In [91]:
Audio(apply_noise(signal, sr, 'clipping'), rate=sr)

In [92]:
files = [str(file) for file in Path('data').rglob('*') if file.is_file()]

In [93]:
noises = []
for path in tqdm(files):
    new_path = 'noised_' + path
    signal, sr = librosa.load(path, sr=None)
    noise = random.choice(['white', 'blue', 'violet', 'brownian', 'pink', 'impulse', 'frequency', 'clipping'])
    noises.append(noise)
    new_signal = apply_noise(signal, sr, noise)
    torchaudio.save(new_path, torch.Tensor(new_signal).unsqueeze(0), sr)

with open("noises.txt", "w") as file:
    for element in noises:
        file.write(element + "\n")

100%|██████████| 1439/1439 [00:58<00:00, 24.60it/s]
